In [24]:
import tensorflow as tf
tf.__version__

import numpy as np

AUTOTUNE = tf.data.AUTOTUNE

In [25]:
import os
from pathlib import Path

project_name = "chest-predictor"
username = os.environ.get('USER')

#Set variables (origin url, destination folder, folder name)
data_url = 'https://storage.googleapis.com/chest-predictor-data/resized_dataset.zip'
data_dir = f"/Users/{username}/code/sachamagier/{project_name}/raw_data"
data_fname = 'resized_dataset.zip'


#Downloading the dataset
data_root_orig = tf.keras.utils.get_file(
    fname=os.path.join(data_dir, data_fname),
    origin=data_url
)

In [26]:
import zipfile

#Extract the dataset
with zipfile.ZipFile(data_root_orig, 'r') as zip_ref:
    zip_ref.extractall(data_dir)


In [27]:
#Set the new data_root
data_root = Path(os.path.join(data_dir, 'resized_dataset'))
print(data_root)

/Users/sachamagier/code/sachamagier/chest-predictor/raw_data/resized_dataset


In [30]:
all_image_paths = [str(path) for path in (data_root/"images"/"set_full").iterdir()]

In [31]:
image_count = len(all_image_paths)
image_count

112120

In [80]:
from PIL import Image

# specify your dataset directory
data_directory = data_root/"images"/"set_full"

# list to store the names of the images to be removed
images_to_remove = []

# iterate over all files in the dataset directory
for filename in os.listdir(data_directory):
    if filename.endswith(".png"):  # make sure it's a png file
        file_path = os.path.join(data_directory, filename)
        image = Image.open(file_path)  # load the image with Pillow

        # convert the image to a numpy array to check its shape
        image_array = np.array(image)

        # check the shape of the image
        if image_array.shape == (256, 256, 4):
            images_to_remove.append(filename)  # add the filename to the list if it's to be removed

# print the names of the images to be removed
print("Images to be removed: ", images_to_remove)


Images to be removed:  ['00027784_006.png', '00000497_000.png', '00030323_020.png', '00007188_006.png', '00006993_000.png', '00004438_004.png', '00001280_003.png', '00006214_000.png', '00004281_003.png', '00007041_000.png', '00003215_000.png', '00004770_000.png', '00005496_000.png', '00001882_001.png', '00007174_001.png', '00028661_000.png', '00000122_000.png', '00001305_000.png', '00005722_002.png', '00000218_004.png', '00000579_000.png', '00007471_002.png', '00001055_002.png', '00014486_003.png', '00006763_003.png', '00004784_000.png', '00014419_001.png', '00003976_000.png', '00030798_000.png', '00001252_001.png', '00005813_000.png', '00000918_000.png', '00005394_000.png', '00021131_005.png', '00004072_000.png', '00003821_001.png', '00005568_000.png', '00004877_001.png', '00014426_000.png', '00005123_000.png', '00011963_001.png', '00000955_001.png', '00001532_000.png', '00003923_007.png', '00003923_005.png', '00007048_000.png', '00000210_000.png', '00001558_010.png', '00006109_000.pn

In [81]:
for filename in images_to_remove:
    file_path = os.path.join(data_directory, filename)
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"{filename} deleted")
    else:
        print("The file does not exist")

00027784_006.png deleted
00000497_000.png deleted
00030323_020.png deleted
00007188_006.png deleted
00006993_000.png deleted
00004438_004.png deleted
00001280_003.png deleted
00006214_000.png deleted
00004281_003.png deleted
00007041_000.png deleted
00003215_000.png deleted
00004770_000.png deleted
00005496_000.png deleted
00001882_001.png deleted
00007174_001.png deleted
00028661_000.png deleted
00000122_000.png deleted
00001305_000.png deleted
00005722_002.png deleted
00000218_004.png deleted
00000579_000.png deleted
00007471_002.png deleted
00001055_002.png deleted
00014486_003.png deleted
00006763_003.png deleted
00004784_000.png deleted
00014419_001.png deleted
00003976_000.png deleted
00030798_000.png deleted
00001252_001.png deleted
00005813_000.png deleted
00000918_000.png deleted
00005394_000.png deleted
00021131_005.png deleted
00004072_000.png deleted
00003821_001.png deleted
00005568_000.png deleted
00004877_001.png deleted
00014426_000.png deleted
00005123_000.png deleted


In [82]:
print(images_to_remove)

['00027784_006.png', '00000497_000.png', '00030323_020.png', '00007188_006.png', '00006993_000.png', '00004438_004.png', '00001280_003.png', '00006214_000.png', '00004281_003.png', '00007041_000.png', '00003215_000.png', '00004770_000.png', '00005496_000.png', '00001882_001.png', '00007174_001.png', '00028661_000.png', '00000122_000.png', '00001305_000.png', '00005722_002.png', '00000218_004.png', '00000579_000.png', '00007471_002.png', '00001055_002.png', '00014486_003.png', '00006763_003.png', '00004784_000.png', '00014419_001.png', '00003976_000.png', '00030798_000.png', '00001252_001.png', '00005813_000.png', '00000918_000.png', '00005394_000.png', '00021131_005.png', '00004072_000.png', '00003821_001.png', '00005568_000.png', '00004877_001.png', '00014426_000.png', '00005123_000.png', '00011963_001.png', '00000955_001.png', '00001532_000.png', '00003923_007.png', '00003923_005.png', '00007048_000.png', '00000210_000.png', '00001558_010.png', '00006109_000.png', '00014223_008.png',

In [83]:
#remove the images from the list in all image paths
all_image_paths = [path for path in all_image_paths if path.split("/")[-1] not in images_to_remove]

In [35]:
#extracting the labels from the csv file
import pandas as pd
# Load labels
labels_df = pd.read_csv(data_root/"Data_Entry_2017.csv")
labels_df.set_index('Image Index', inplace=True)

In [84]:
image_count = len(all_image_paths)
image_count

111601

In [118]:
import pandas as pd
# Load labels
labels_df = pd.read_csv(data_root/"Data_Entry_2017.csv")
labels_df.set_index('Image Index', inplace=True)

In [120]:
#remove in labels_df the images that have been removed
labels_df = labels_df.drop(images_to_remove)


In [121]:
#len of the dataframe
len(labels_df)

111601

In [122]:
print(labels_df)

                          Finding Labels  Follow-up #  Patient ID  \
Image Index                                                         
00000001_000.png            Cardiomegaly            0           1   
00000001_001.png  Cardiomegaly|Emphysema            1           1   
00000001_002.png   Cardiomegaly|Effusion            2           1   
00000002_000.png              No Finding            0           2   
00000003_000.png                  Hernia            0           3   
...                                  ...          ...         ...   
00030801_000.png              No Finding            0       30801   
00030801_001.png          Mass|Pneumonia            1       30801   
00030802_000.png              No Finding            0       30802   
00030803_000.png              No Finding            0       30803   
00030804_000.png              No Finding            0       30804   

                  Patient Age Patient Gender View Position  \
Image Index                             

In [123]:
#check the porcentage of 1 or more label in the dataframe
labels_df['Finding Labels'].str.contains('\|').sum() / len(labels_df)


0.18575102373634644

In [124]:
#remove the no finding label from the dataframe
labels_df = labels_df[labels_df['Finding Labels'] != 'No Finding']


In [125]:
#check the number of each label
labels_df['Finding Labels'].value_counts()


Finding Labels
Infiltration                                                      9522
Atelectasis                                                       4207
Effusion                                                          3940
Nodule                                                            2688
Pneumothorax                                                      2186
                                                                  ... 
Consolidation|Effusion|Infiltration|Mass|Pneumothorax                1
Atelectasis|Effusion|Emphysema|Mass|Pneumothorax                     1
Edema|Infiltration|Mass|Nodule|Pneumonia                             1
Consolidation|Emphysema|Mass|Pneumothorax                            1
Atelectasis|Consolidation|Mass|Pleural_Thickening|Pneumothorax       1
Name: count, Length: 835, dtype: int64

In [126]:
#remove all the multiple labels
labels_df['Finding Labels'] = labels_df['Finding Labels'].apply(lambda x: x.split("|")[0])


In [127]:
#check the percentage of each label with the new dataset (without no finding and multiple labels)
labels_df['Finding Labels'].value_counts(normalize=True)


Finding Labels
Infiltration          0.228090
Atelectasis           0.223088
Effusion              0.155297
Consolidation         0.064399
Nodule                0.058001
Mass                  0.056509
Cardiomegaly          0.046467
Pneumothorax          0.042493
Edema                 0.035941
Emphysema             0.033498
Pleural_Thickening    0.023495
Fibrosis              0.023398
Pneumonia             0.006300
Hernia                0.003024
Name: proportion, dtype: float64

In [128]:
#now print the absolute value of the labels
labels_df['Finding Labels'].value_counts()


Finding Labels
Infiltration          11766
Atelectasis           11508
Effusion               8011
Consolidation          3322
Nodule                 2992
Mass                   2915
Cardiomegaly           2397
Pneumothorax           2192
Edema                  1854
Emphysema              1728
Pleural_Thickening     1212
Fibrosis               1207
Pneumonia               325
Hernia                  156
Name: count, dtype: int64

In [129]:
#remove pneumonia and hernia from the dataframe
labels_df = labels_df[labels_df['Finding Labels'] != 'Pneumonia']
labels_df = labels_df[labels_df['Finding Labels'] != 'Hernia']


In [130]:
labels_df['Finding Labels'].value_counts()


Finding Labels
Infiltration          11766
Atelectasis           11508
Effusion               8011
Consolidation          3322
Nodule                 2992
Mass                   2915
Cardiomegaly           2397
Pneumothorax           2192
Edema                  1854
Emphysema              1728
Pleural_Thickening     1212
Fibrosis               1207
Name: count, dtype: int64

In [132]:
#in label df make all the finding labels having the same number of images (the number of the smallest label)
min_label = labels_df['Finding Labels'].value_counts().min()
labels_df = labels_df.groupby('Finding Labels').head(min_label)

labels_df['Finding Labels'].value_counts()


Finding Labels
Cardiomegaly          1207
Infiltration          1207
Effusion              1207
Nodule                1207
Emphysema             1207
Atelectasis           1207
Pleural_Thickening    1207
Pneumothorax          1207
Mass                  1207
Fibrosis              1207
Consolidation         1207
Edema                 1207
Name: count, dtype: int64

In [133]:
#create a new csv file with the new labels
labels_df.to_csv(data_root/"Data_Entry_2017_clean.csv")
